In [20]:
import os

import time
import json

from dotenv import load_dotenv

import random

import google.generativeai as genai
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

In [21]:
load_dotenv()

SCOPES = ["https://www.googleapis.com/auth/cloud-platform", "https://www.googleapis.com/auth/generative-language.tuning"]
KEY_FILE = os.getenv("PARENT_DIR") + "auth/credentials.json"
TOKEN_FILE = os.getenv("PARENT_DIR") + "auth/token.json"

In [22]:
creds = None

if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(KEY_FILE, SCOPES)
        creds = flow.run_local_server()
    
    with open(TOKEN_FILE, "w") as token:
        token.write(creds.to_json())

In [23]:
genai.configure(credentials=creds)

In [24]:
model = genai.GenerativeModel("gemini-1.5-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [25]:
base_model = [m for m in genai.list_models() if "createTunedModel" in m.supported_generation_methods][0]
base_model 

Model(name='models/gemini-1.0-pro-001',
      base_model_id='',
      version='001',
      display_name='Gemini 1.0 Pro 001 (Tuning)',
      description=('The best model for scaling across a wide range of tasks. This is a stable '
                   'model that supports tuning.'),
      input_token_limit=30720,
      output_token_limit=2048,
      supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
      temperature=0.9,
      max_temperature=None,
      top_p=1.0,
      top_k=None)

In [26]:
with open(os.getenv("PARENT_DIR") + "data/final_data.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())

    training_data = [{"text_input": a, "output": b} for a, b in data]

training_data[:5]

[{'text_input': 'What is your name?', 'output': 'My name is The Orientator'},
 {'text_input': 'What do you do?',
  'output': 'I am an AI Chatbot named The Orientator, and my purpose is to help you understand HCI better.'},
 {'text_input': 'How can you help me?',
  'output': 'You can ask me questions about the HCI, and I will try my best to help you answer them.'},
 {'text_input': 'Who are you?',
  'output': 'I am an AI Chatbot named The Orientator, and my purpose is to help you understand HCI better.'},
 {'text_input': 'What are you?',
  'output': 'I am an AI Chatbot named The Orientator, and my purpose is to help you understand HCI better.'}]

In [27]:
name = "test-model-7"
operation = genai.create_tuned_model(
    source_model=base_model.name,
    training_data=training_data,
    id = name,
    epoch_count = 100,
    batch_size = 32,
    learning_rate = 1e-2,
)

In [28]:
for status in operation.wait_bar():
    time.sleep(2)

  0%|          | 0/2360 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model = genai.get_tuned_model(f"tunedModels/{name}")
model

TunedModel(name='tunedModels/test-model-6',
           source_model='models/gemini-1.5-flash-001-tuning',
           base_model='models/gemini-1.5-flash-001-tuning',
           display_name='',
           description='',
           temperature=1.0,
           top_p=0.95,
           top_k=64,
           state=<State.CREATING: 1>,
           create_time=datetime.datetime(2024, 8, 12, 5, 48, 20, 826545, tzinfo=datetime.timezone.utc),
           update_time=datetime.datetime(2024, 8, 12, 5, 48, 20, 826545, tzinfo=datetime.timezone.utc),
           tuning_task=TuningTask(start_time=None,
                                  complete_time=None,
                                  snapshots=[],
                                  hyperparameters=Hyperparameters(epoch_count=100,
                                                                  batch_size=32,
                                                                  learning_rate=0.01)))

In [ ]:
model = genai.GenerativeModel(model_name=f"tunedModels/{name}")
model

genai.GenerativeModel(
    model_name='tunedModels/test-model-2',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [ ]:
content = model.generate_content("What can I do leisurely in HCI?")
print(content.text)

**Fun things to do in HCI (non-exhaustive)**

* **Basketball** (Note: Do note that the courts are very popular and you may not be able to play on it)
* **Table tennis** (Note: Do note that the tables are very popular and you may not be able to play on it)
* **Going to the track to run/jog** (Note: Do note that track and field CCA teams use the track and you may not be able to use it)
* **Going to the gym (Note: Do note that you need to pay to use it)
* **Chilling at the track/EP3 (Note: Do note that track and field CCA teams use the track and you may not be able to use it)
* **Going to the library to study/find a book to read (Note: Do note that you need to keep quiet as people are studying there)
* **Going to the darkroom to develop photos (Note: Do note that you need to pay to use it)
* **Playing the piano at the Clocktower/LT400 (Note: Do note that you need to check if there are any events on that day before playing the piano)
* **Taking photos at the track/EP3/HC (Note: Do note tha